In [1]:
# Standard library imports
import os
import requests

# Third party imports
import ee
import ee.mapclient
import folium
import geopandas as gpd
from google.oauth2 import service_account
from osgeo import gdal
from pyproj import Transformer

from dotenv import load_dotenv
import rasterio
import numpy as np

#Specify the path to the service account key file (found in .env)
load_dotenv()
key_path = os.getenv('GEE_AUTH_KEY_PATH')

# Define the correct scope for Earth Engine
SCOPES = ['https://www.googleapis.com/auth/earthengine']

# Load the service account credentials
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=SCOPES
)

# Initialize Earth Engine
ee.Initialize(credentials=credentials)
print("Earth Engine initialized successfully!")

Earth Engine initialized successfully!


In [2]:

roi = 'data/vector/all_areas.geojson'
buffer_amt = 2500

# Read the ROI file
gdf = gpd.read_file(roi)

# Create a buffered envelope (bounding box)
buffered_bbox = gdf.envelope.buffer(buffer_amt, cap_style=3)  # cap_style=3 creates square buffers

# Transform to EPSG:4326 for Earth Engine
buffered_bbox_4326 = buffered_bbox.to_crs("EPSG:4326")

# Convert the GeoSeries to a GeoDataFrame
buffered_bbox_gdf = gpd.GeoDataFrame(geometry=buffered_bbox_4326)

# Convert all geometries to Earth Engine geometries
ee_geometries = []
for _, row in buffered_bbox_gdf.iterrows():
    coords = row.geometry.exterior.coords.xy
    ee_coords = [[x, y] for x, y in zip(coords[0], coords[1])]
    ee_geometries.append(ee.Geometry.Polygon(ee_coords, proj='EPSG:4326'))

# Create a GeometryCollection from the list of geometries
geometry_collection = ee.Geometry.MultiPolygon(ee_geometries)

# Get the bounding box of the GeometryCollection
bounding_box = geometry_collection.bounds()
# Image collection path
path_to_asset = 'projects/rap-data-365417/assets/vegetation-cover-v3'

# Load the image collection
image_collection = ee.ImageCollection(path_to_asset)

available_years = [
    1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995,
    1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
    2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
    2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023
]

# Initialize woodyCover as None
woodyCover = None

# Process each year in the list
for year in available_years:
    # Filter images for the specific year
    yearly_image = image_collection.filter(
        ee.Filter.calendarRange(year, year, 'year')
    ).median().clip(bounding_box)

    # Calculate woody cover (tree + shrub cover) - Corrected band selection
    treeCover = yearly_image.select('TRE').rename(f'tree_cover_{year}')
    shrubCover = yearly_image.select('SHR').rename(f'shrub_cover_{year}')

    woodyCoverYear = treeCover.add(shrubCover).rename(f'woody_cover_{year}')
    # Add the yearly woody cover as a new band - Handling the first year differently
    if woodyCover is None:
        woodyCover = woodyCoverYear
    else:
        woodyCover = woodyCover.addBands(woodyCoverYear)

# Check the final woodyCover image bands
print("Final woodyCover bands:", woodyCover.bandNames().getInfo())

Final woodyCover bands: ['woody_cover_1986', 'woody_cover_1987', 'woody_cover_1988', 'woody_cover_1989', 'woody_cover_1990', 'woody_cover_1991', 'woody_cover_1992', 'woody_cover_1993', 'woody_cover_1994', 'woody_cover_1995', 'woody_cover_1996', 'woody_cover_1997', 'woody_cover_1998', 'woody_cover_1999', 'woody_cover_2000', 'woody_cover_2001', 'woody_cover_2002', 'woody_cover_2003', 'woody_cover_2004', 'woody_cover_2005', 'woody_cover_2006', 'woody_cover_2007', 'woody_cover_2008', 'woody_cover_2009', 'woody_cover_2010', 'woody_cover_2011', 'woody_cover_2012', 'woody_cover_2013', 'woody_cover_2014', 'woody_cover_2015', 'woody_cover_2016', 'woody_cover_2017', 'woody_cover_2018', 'woody_cover_2019', 'woody_cover_2020', 'woody_cover_2021', 'woody_cover_2022', 'woody_cover_2023']


In [3]:

# Define visualization parameters
vis_params = {'min': 0, 'max': 100, 'palette': ['440154', '482878', '3E4A89', '31688E', '26828E', '1F9E89', '35B779', '6DCD59', 'B4DE2C', 'FDE725']}

# Create a folium map object.
# Instead of using centroid, use the bounding box coordinates directly
bounding_box_coords = bounding_box.coordinates().getInfo()[0]
# Calculate the center of the bounding box
lon_coords = [coord[0] for coord in bounding_box_coords]
lat_coords = [coord[1] for coord in bounding_box_coords]
centroid_lon = (min(lon_coords) + max(lon_coords)) / 2
centroid_lat = (min(lat_coords) + max(lat_coords)) / 2

m = folium.Map(location=[centroid_lat, centroid_lon], zoom_start=12)

# List of years to display
years_to_display = [1986, 2023]

for year in years_to_display:
    # Get the band name for the current year
    woody_band_name = f'woody_cover_{year}'

    # Check if the band exists
    if woody_band_name in woodyCover.bandNames().getInfo():
        # Select the woody cover band for the current year
        woodyCover_year = woodyCover.select(woody_band_name)

        # Add the image layer to the map
        map_id_dict = woodyCover_year.getMapId(vis_params)
        folium.TileLayer(
            tiles=map_id_dict['tile_fetcher'].url_format,
            attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
            overlay=True,
            name=f'woody_cover_{year}',
        ).add_to(m)
    else:
        print(f"Band '{woody_band_name}' not found in woodyCover.")

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map
display(m)

In [26]:
rap_dir = 'data/raster/rap'
os.makedirs(rap_dir, exist_ok=True)

# Set the GTIFF_SRS_SOURCE configuration option to EPSG
gdal.SetConfigOption('GTIFF_SRS_SOURCE', 'EPSG')

# Assuming 'woodyCover' is your ee.Image and 'bounding_box' is your ee.Geometry

for cover_image in [woodyCover]:
    for band_name in cover_image.bandNames().getInfo():
        # Choose a value that doesn't exist in your data and doesn't have meaning in the context of your data.
        remapped_image = cover_image.select(band_name).unmask(255)

        # Set an explicit "no data" value to null in the download URL parameters
        url = remapped_image.getDownloadURL({
            'scale': 30,
            'crs': 'EPSG:6514',
            'region': bounding_box,
            'format': 'GeoTIFF',
            'formatOptions': {
                'cloudOptimized': True,
                'noData': 255
            }
        })

        response = requests.get(url)
        
        # Create output file path
        output_file = os.path.join(rap_dir, f"{band_name}.tif")
        
        # Write response content directly to final file
        with open(output_file, 'wb') as f:
            f.write(response.content)
        
        print(f"Downloaded file {output_file}")

        # Set nodata value to 255
        dataset = gdal.Open(output_file, gdal.GA_Update)
        band = dataset.GetRasterBand(1)
        band.SetNoDataValue(255)
        dataset = None

Downloaded file data/raster/rap/woody_cover_1986.tif


Warning 1: TIFFWriteDirectorySec:Creating TIFF with legacy Deflate codec identifier, COMPRESSION_ADOBE_DEFLATE is more widely supported


Downloaded file data/raster/rap/woody_cover_1987.tif


Warning 1: TIFFWriteDirectorySec:Creating TIFF with legacy Deflate codec identifier, COMPRESSION_ADOBE_DEFLATE is more widely supported


Downloaded file data/raster/rap/woody_cover_1988.tif


...

Warning 1: TIFFWriteDirectorySec:Creating TIFF with legacy Deflate codec identifier, COMPRESSION_ADOBE_DEFLATE is more widely supported


Downloaded file data/raster/rap/woody_cover_2022.tif


Warning 1: TIFFWriteDirectorySec:Creating TIFF with legacy Deflate codec identifier, COMPRESSION_ADOBE_DEFLATE is more widely supported


Downloaded file data/raster/rap/woody_cover_2023.tif


Warning 1: TIFFWriteDirectorySec:Creating TIFF with legacy Deflate codec identifier, COMPRESSION_ADOBE_DEFLATE is more widely supported


In [23]:
# Loop through the years
for year in range(1986, 2024):
    # Construct file paths for woody and herb cover
    woody_file = f'woody_cover_{year}.tif'
    herb_file = f'herb_cover_{year}.tif'
    woody_path = os.path.join(rap_dir, woody_file)
    herb_path = os.path.join(rap_dir, herb_file)

    # Check if both files exist
    if os.path.exists(woody_path) and os.path.exists(herb_path):
        with rasterio.open(woody_path) as src_woody, rasterio.open(herb_path) as src_herb:
            # Read the nodata value from the metadata
            woody_nodata = src_woody.nodata
            herb_nodata = src_herb.nodata

            # Read data as arrays
            woody_array = src_woody.read(1)
            herb_array = src_herb.read(1)

            # Mask out no-data values
            woody_array_masked = np.ma.masked_where(woody_array == woody_nodata, woody_array)
            herb_array_masked = np.ma.masked_where(herb_array == herb_nodata, herb_array)

            # Flatten the arrays and remove masked values (no-data values)
            woody_flat = woody_array_masked.compressed()
            herb_flat = herb_array_masked.compressed()

            # Calculate correlation if there is valid data
            if woody_flat.size > 0 and herb_flat.size > 0:
                correlation = np.corrcoef(woody_flat, herb_flat)[0, 1]
                print(f"Correlation between woody and herb cover for year {year}: {correlation:.2f}")
            else:
                print(f"No valid data for correlation calculation for year {year}")
    else:
        print(f"Missing data for year {year} (either woody or herb cover file is missing)")

Correlation between woody and herb cover for year 1986: -0.64
Correlation between woody and herb cover for year 1987: -0.57
Correlation between woody and herb cover for year 1988: -0.58
Correlation between woody and herb cover for year 1989: -0.52
Correlation between woody and herb cover for year 1990: -0.52
Correlation between woody and herb cover for year 1991: -0.56
Correlation between woody and herb cover for year 1992: -0.48
Correlation between woody and herb cover for year 1993: -0.53
Correlation between woody and herb cover for year 1994: -0.69
Correlation between woody and herb cover for year 1995: -0.63
Correlation between woody and herb cover for year 1996: -0.66
Correlation between woody and herb cover for year 1997: -0.54
Correlation between woody and herb cover for year 1998: -0.63
Correlation between woody and herb cover for year 1999: -0.65
Correlation between woody and herb cover for year 2000: -0.60
Correlation between woody and herb cover for year 2001: -0.69
Correlat